In [ ]:
source("../shared.R")

In [ ]:
data("warpbreaks")

In [ ]:
?warpbreaks

In [ ]:
head(warpbreaks)

In [ ]:
table(warpbreaks$wool, warpbreaks$tension)

In [ ]:
boxplot(log(breaks) ~ wool + tension, data = warpbreaks)

Different groups have mor similar simulation if we use $log(.)$ of the breaks. And we use it.

## One way model, where groups are split as tension

In [ ]:
str(warpbreaks)

In [ ]:
smod = "model {
    for (i in 1: length(y)) {
        y[i] ~ dnorm(mu[tens_grp[i]], prec)
    }
    
    for (j in 1:3) {
        mu[j] ~ dnorm(0, 1./1e6) # low informative prior
    }
    
    prec ~ dgamma(5/2.0, 5*2/2.0) # 5 - sample size, 2 variation
    sig = sqrt(1./prec)  
}"

set.seed(83)

In [ ]:
data_ =  list(y = log(warpbreaks$breaks), tens_grp = as.numeric(warpbreaks$tension))
mod = jags_model_compile(str_model_ = smod, data_ = data_, n_chains_ = 3, burnout = 1e3)
mod_sim = jags_model_sample(mod, var_names_ = c("mu", "sig"), n_iter_ = 1e4)
mod_diag = jags_model_diag(mod_sim$samples)

In [ ]:
plot(mod_sim$samples)

In [ ]:
dic1 = dic.samples(model = mod, n.iter = 1e4)

#### From the simulation we get the same results as we get them for just averaging them out

that is an expected behaviour

In [ ]:
aggregate(log(warpbreaks$breaks), list(warpbreaks$tension), mean)

In [ ]:
colMeans(mod_sim$samples_cmb)

In [ ]:
summary(mod_sim$samples_cmb)

### Two-way additive model (additive?)

In [ ]:
X = model.matrix(~ wool + tension, data=warpbreaks)

In [ ]:
smod = "model {
    for (i in 1:length(y)) {
        y[i] ~ dnorm(mu[i], prec)
        mu[i] = int + alpha*isWoolB[i] + beta[1]*isTeninsionM[i] + beta[2]*isTensionH[i]    
    }
    int ~ dnorm(0, 1.e-6)
    alpha ~ dnorm(0, 1.e-6)
    for (j in 1:2) {
        beta[j] ~ dnorm(0, 1.e-6)
    }
    prec ~ dgamma(3/2., 3.2)
    sig = sqrt(1/prec)
}"

In [ ]:
dat = list(y=log(warpbreaks$breaks), isWoolB=X[,"woolB"], isTeninsionM=X[,"tensionM"], isTensionH=X[,"tensionH"]) 

In [ ]:
jags_mod = jags_model_compile(smod, dat)
mod_sim = jags_model_sample(jags_mod, n_iter_ = 1e4, var_names_ = c("int", "alpha","beta", "sig"))
mod_diag = jags_model_diag(mod_sim$samples)

In [ ]:
smod = "model {
    for (i in 1:length(y)) {
        y[i] ~ dnorm(mu[wool_group[i], tension_group[i]], prec[wool_group[i], tension_group[i]] )
    }
    # prec ~ dgamma(3/2.0, 3*1/2.9)
    # sig = sqrt(1/prec)
    for (j in 1:max(wool_group)) {
        for (k in 1:max(tension_group)) {
            mu[j,k] ~ dnorm(0, 1e-6)
            prec[j,k] ~ dgamma(1/2, 1/2)
            sig[j, k] = sqrt(1/prec[j,k])
        }
    }
}"
dat = list(y = log(warpbreaks$breaks), wool_group=as.numeric(warpbreaks$wool), tension_group=as.numeric(warpbreaks$tension) )
jags_mod = jags_model_compile(smod, dat)
mod_sim = jags_model_sample(jags_mod, var_names_ =c("mu", "sig"),n_iter_ = 1e4)
mod_diag = jags_model_diag(samples_ = mod_sim$samples)